In [13]:
from psaw import PushshiftAPI
from pathlib import Path 
# from datetime import datetime, timezone, timedelta  ## incase timezone modifcations are needed
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import os
import glob
import dask.dataframe as dd
import math

PAR_LENGTH = 150 # in number of words
FILTER_REQ = 0.7 # evaluation value for number of paragraphs

In [14]:
files = glob.glob(r"C:\Users\MoRevolution\Desktop\College\P&R\Research_LowSES\Data Collection\Reddit Search Keywords/*.txt", 
                   recursive = True)

for f in files: 
    print(f)

filenames = []
for i in range(len(files)): 
    filenames.append(files[i][files[i].rfind("\\")+1:-4])

for f in filenames: 
    print(f)

C:\Users\MoRevolution\Desktop\College\P&R\Research_LowSES\Data Collection\Reddit Search Keywords\Manual Search Keywords.txt
C:\Users\MoRevolution\Desktop\College\P&R\Research_LowSES\Data Collection\Reddit Search Keywords\Modified Keywords .txt
C:\Users\MoRevolution\Desktop\College\P&R\Research_LowSES\Data Collection\Reddit Search Keywords\Modified Keywords.txt
C:\Users\MoRevolution\Desktop\College\P&R\Research_LowSES\Data Collection\Reddit Search Keywords\SVO Search Keywords.txt
Manual Search Keywords
Modified Keywords 
Modified Keywords
SVO Search Keywords


In [15]:
api = PushshiftAPI()

In [16]:
keyword_list = []
master_keyword_list = []
filename_counter = 0

for f in files: 
    with open(f, 'r') as file:
        keyword_list = []
        for line in file:
            keyword_list.append(line.strip('\n')) 
            
    keyword_list = [i for i in keyword_list if i]
    master_keyword_list.append(keyword_list)

print(master_keyword_list)

[["can't pay for school", 'struggling with tuition', 'rejected because of finances when applying to college', 'imposer syndrome in college ', 'imposter syndrome in a competitive highschool', 'school is too expensive', "can't afford tuition ", 'first-gen college student guilt ', 'first-gen college student family pressure', 'first-gen college student struggling with finances', 'lack of emotional support from parents in college', "my family can't afford to pay school for me ", "i can't afford a dorm"], ["tried hard in school OR dropped out school OR I was the poorest kid in class OR i can't afford a dorm", "i can't afford a dorm OR school is too expensive OR can't pay for school OR first-gen college student struggling with finance", 'growing in poor family OR I come from low income family of substance abusers OR grew up with single mum OR grew up with single father', 'growing in poor family OR I come from low income family of substance abusers OR first-gen college student family pressure'

In [18]:
# Suggestions
    ## write a script that tries out multiple word combinations for all of the keywords already provided 
    ## consider using cudf instead of pandas for faster dataframe processing
    ## test this code using dask for CPU parallel computing
subreddit_list = ["college", "AskReddit", "science", "psychology", "socialwork", "personalfinance", "financialaid"]
for keyword_n in master_keyword_list:
    count = 0 
    for keywords in keyword_list: 
        for subreddit_n in subreddit_list:   
            subs = api.search_submissions(
                subreddit= subreddit_n,
                q= keywords,
                filter = [ 'author', 'selftext', 'url', 'permalink', 'subreddit', 'title'],
                # metadata = "false",   
                # max_results_per_request= 500
                )

            # Set display
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_colwidth', None)

            df = pd.DataFrame([thing.d_ for thing in subs])
            ddf = dd.from_pandas(df, npartitions=10)

            ddf["Number of Paragraphs"] = ddf["body"].apply(lambda n: len(n.split())/PAR_LENGTH) 
            #filtered data frame with many 
            dff_filtered = ddf[ddf["Number of Paragraphs"] >= FILTER_REQ]

            filepath = Path('Data Dumb/Reddit_1/Submissions/'+ subreddit_n +'/'+filenames[filename_counter]+'/Keyword_'+ str(count)+'.csv')
            filepath.parent.mkdir(parents = True, exist_ok = True)
            ddf.to_csv(filepath)
            count += 1 
    filename_counter += 1 

KeyError: 'body'